In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
pip install wandb

In [3]:
import wandb

In [4]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
# Langkah 2: Import Libraries
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, models
from datasets import load_dataset
import wandb

In [6]:
# Langkah 3: Inisialisasi WandB
wandb.login()  # Pastikan Anda login dengan akun WandB
wandb.init(project="trashnet_classification", config={
    "epochs": 10,
    "batch_size": 32,
    "learning_rate": 0.001,
    "architecture": "ResNet18"
})

wandb: Currently logged in as: sintongarga (sintongarga-binus-university). Use `wandb login --relogin` to force relogin


In [7]:
# Langkah 4: Load Dataset TrashNet
ds = load_dataset("garythung/trashnet", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

dataset-original.zip:   0%|          | 0.00/3.63G [00:00<?, ?B/s]

dataset-resized.zip:   0%|          | 0.00/42.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5054 [00:00<?, ? examples/s]

In [8]:
# Langkah 5: Preprocessing dan Transformasi Dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize gambar agar sesuai dengan input model
    transforms.ToTensor(),          # Konversi ke tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalisasi
])

class TrashDataset(torch.utils.data.Dataset):
    def __init__(self, ds, transform=None):
        self.ds = ds
        self.transform = transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        image = self.ds[idx]['image']
        label = self.ds[idx]['label']
        if self.transform:
            image = self.transform(image)
        return image, label

# Dataset dengan Transformasi
dataset = TrashDataset(ds, transform=transform)

In [9]:
# Langkah 6: Split Dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = wandb.config["batch_size"]
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [10]:
# Langkah 7: Setup Model (ResNet18 Pretrained)
model = models.resnet18(pretrained=True)
num_classes = len(set(ds['label']))
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Gunakan GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 173MB/s]


In [11]:
# Langkah 8: Define Loss dan Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=wandb.config["learning_rate"])

In [13]:
# Langkah 9: Training Loop
num_epochs = wandb.config["epochs"]

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    wandb.log({"epoch": epoch + 1, "train_loss": avg_train_loss})
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_train_loss:.4f}")

    # Evaluation Loop
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    wandb.log({"epoch": epoch + 1, "validation_accuracy": accuracy})
    print(f"Validation Accuracy: {accuracy:.2f}%")

Epoch 1/10, Loss: 0.8915
Validation Accuracy: 70.92%
Epoch 2/10, Loss: 0.5399
Validation Accuracy: 75.87%
Epoch 3/10, Loss: 0.4175
Validation Accuracy: 74.98%
Epoch 4/10, Loss: 0.3255
Validation Accuracy: 78.14%
Epoch 5/10, Loss: 0.2720
Validation Accuracy: 85.16%
Epoch 6/10, Loss: 0.2041
Validation Accuracy: 82.89%
Epoch 7/10, Loss: 0.1748
Validation Accuracy: 85.66%
Epoch 8/10, Loss: 0.2100
Validation Accuracy: 82.99%
Epoch 9/10, Loss: 0.1151
Validation Accuracy: 87.14%
Epoch 10/10, Loss: 0.0988
Validation Accuracy: 84.37%


In [14]:
# Langkah 10: Akhiri Logging WandB
wandb.finish()

epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
train_loss,█▅▄▃▃▂▂▂▁▁
validation_accuracy,▁▃▃▄▇▆▇▆█▇
epoch,10
train_loss,0.09876
validation_accuracy,84.37191
